##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pose Landmarks Detection with MediaPipe Tasks

This notebook shows you how to use MediaPipe Tasks Python API to detect pose landmarks from images.

## Preparation

Let's start with installing MediaPipe.


In [1]:
!pip install -q mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 15.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


Then download an off-the-shelf model bundle. Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker#models) for more information about this model bundle.

In [2]:
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

## Visualization utilities

In [1]:
#@markdown To better demonstrate the Pose Landmarker API, we have created a set of visualization tools that will be used in this colab. These will draw the landmarks on a detect person, as well as the expected connections between those markers.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

## Download test image

To demonstrate the Pose Landmarker API, you can download a sample image using the follow code. The image is from [Pixabay](https://pixabay.com/photos/girl-woman-fitness-beautiful-smile-4051811/).

In [4]:
!wget -q -O image.jpg https://cdn.pixabay.com/photo/2019/03/12/20/39/girl-4051811_960_720.jpg

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [6]:


import cv2


img = cv2.imread("girl.jpg")
cv2.imshow("image", img)
cv2.waitKey(0)

-1

Optionally, you can upload your own image. If you want to do so, uncomment and run the cell below.

In [ ]:
# from google.colab import files
# uploaded = files.upload()

# for filename in uploaded:
#   content = uploaded[filename]
#   with open(filename, 'wb') as f:
#     f.write(content)

# if len(uploaded.keys()):
#   IMAGE_FILE = next(iter(uploaded))
#   print('Uploaded file:', IMAGE_FILE)

## Running inference and visualizing the results

The final step is to run pose landmark detection on your selected image. This involves creating your PoseLandmarker object, loading your image, running detection, and finally, the optional step of displaying the image with visualizations.

Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker/python) to learn more about configuration options that this solution supports.


In [8]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='pose_landmarker_heavy.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file("girl.jpg")

# STEP 4: Detect pose landmarks from the input image.
detection_result = detector.detect(image)

# STEP 5: Process the detection result. In this case, visualize it.
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
cv2.imshow("detection", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)

c:\Users\ASUS\.conda\envs\gym-ai\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


-1

Visualize the pose segmentation mask.

In [9]:
segmentation_mask = detection_result.segmentation_masks[0].numpy_view()
visualized_mask = np.repeat(segmentation_mask[:, :, np.newaxis], 3, axis=2) * 255
cv2.imshow("Masked Image", visualized_mask)
cv2.waitKey(0)

113

: 

In [33]:
import cv2
import numpy as np

# Function to calculate the intersection of two lines
def line_intersection(line1, line2):
    xdiff = (line1[0] - line1[2], line2[0] - line2[2])
    ydiff = (line1[1] - line1[3], line2[1] - line2[3])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
       raise Exception('Lines do not intersect')

    d = (det((line1[0], line1[1]), (line1[2], line1[3])), det((line2[0], line2[1]), (line2[2], line2[3])))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return int(x), int(y)

# Define two lines (x1, y1, x2, y2)
line1 = (100, 350, 300, 300)
line2 = (400, 100, 300, 300)

# Create a blank image
image = np.zeros((500, 500, 3), dtype=np.uint8)
# image = cv2.imread('girl.jpg')
# Draw the two lines
cv2.line(image, (line1[0], line1[1]), (line1[2], line1[3]), (255, 0, 0), 2)
cv2.line(image, (line2[0], line2[1]), (line2[2], line2[3]), (0, 0, 255), 2)

# Find the intersection point of the two lines
intersection = line_intersection(line1, line2)
print(intersection)
# Calculate the angles of the two lines relative to the intersection point
angle1 = np.arctan2(line1[1] - intersection[1], line1[0] - intersection[0])
angle2 = np.arctan2(line2[1] - intersection[1], line2[0] - intersection[0])

# Convert angles to degrees
angle1 = np.degrees(angle1)
angle2 = np.degrees(angle2)
print(angle1, angle2, intersection)
# Ensure the angle is within the 0-360 degree range for OpenCV
if angle1 < 0:
    angle1 += 360
if angle2 < 0:
    angle2 += 360
# Define the radius of the arc
radius = 100
print(angle1, angle2, intersection)
# Draw the arc between the two lines
cv2.ellipse(image, intersection, (radius, radius), 0, angle1, angle2, (0, 255, 0), 2)

# Show the image
cv2.imshow('Arc between lines', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

(300, 300)
165.96375653207352 -63.43494882292201 (300, 300)
165.96375653207352 296.565051177078 (300, 300)


In [8]:
import math 
m2, m1 = (300-250)/(300-100), (200)/(-100)

angle = math.degrees(math.atan(m2-m1/(1+m1*m2)))
angle

76.7594800848128

In [44]:
import cv2
import numpy as np

# Define two lines (in the form of point1, point2)
line1_start = (100, 350)
line2_start = (400, 100)
line1_end = (300, 300)
line2_end = (300, 300)

# Draw the two lines
image = np.zeros((500, 500, 3), dtype=np.uint8)
cv2.line(image, line1_start, line1_end, (255, 255, 255), 2)
cv2.line(image, line2_start, line2_end, (255, 255, 255), 2)

# Calculate the angle between the two lines
angle1 = np.arctan2(line1_end[1] - line1_start[1], line1_end[0] - line1_start[0])
angle2 = np.arctan2(line2_end[1] - line2_start[1], line2_end[0] - line2_start[0])

# Intersection point is the same for both lines (line1_start and line2_start)
intersection = line1_end

# Define the radius of the arc
radius = 50

# Convert angles from radians to degrees
start_angle = np.degrees(angle1)
end_angle = np.degrees(angle2)
print(angle)

arc_angle = abs(angle2 - angle1)
if arc_angle > 180:
    arc_angle = 360 - arc_angle

start_angle = min(angle1, angle2)
end_angle = start_angle + arc_angle
# Ensure the angle is within the 0-360 degree range for OpenCV
# if start_angle < 0:
#     start_angle += 360
# if end_angle < 0:
#     end_angle += 360

# Draw the arc using the intersection point, radius, and angles
cv2.ellipse(image, intersection, (radius, radius), 0, start_angle, end_angle, (0, 255, 0), 2)

# Display the image
cv2.imshow("Arc between lines", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

76.7594800848128


In [13]:
import cv2
import numpy as np
import math

# Define the points for the two lines
line1_start = (100, 350)
line1_end = (300, 300)
line2_start = (400, 100)
line2_end = (300, 300)

# Create a blank image
image = np.zeros((500, 500, 3), dtype=np.uint8)

# Draw the two lines
cv2.line(image, line1_start, line1_end, (255, 0, 0), 2)
cv2.line(image, line2_start, line2_end, (0, 255, 0), 2)

# Calculate angles of the two lines relative to the horizontal
angle1 = math.atan2(line1_start[1] - line1_end[1], line1_start[0] - line1_end[0])
angle2 = math.atan2(line2_start[1] - line2_end[1], line2_start[0] - line2_end[0])
# Convert angles from radians to degrees
angle1_deg = np.degrees(angle1)
angle2_deg = np.degrees(angle2)
print(angle1, angle2, angle1_deg, angle2_deg)

# Ensure the angles are positive and lie within 0 to 360 degrees
# angle1_deg = angle1_deg % 360
# angle2_deg = angle2_deg % 360

# Calculate the internal angle between the two lines
arc_angle = abs(angle2_deg - angle1_deg)
print("arc_angle: ", arc_angle)
if arc_angle > 180:
    arc_angle = 360 - arc_angle  # Use the smaller angle

# Define the center of the arc (intersection point of the lines)
center = line1_end  # Intersection at (300, 300)

# Define radius for the arc
radius = 50

# Determine the start and end angles for the arc
start_angle = max(angle1_deg, angle2_deg)
end_angle = start_angle + arc_angle
print(f" Start Angle: {start_angle} End Angle: {end_angle}")

# Draw the arc
cv2.ellipse(image, center, (radius, radius), 0, start_angle, end_angle, (0, 255, 255), 2)

# Show the image
cv2.imshow('Arc between lines', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


2.896613990462929 -1.1071487177940904 165.96375653207352 -63.43494882292201
arc_angle:  229.3987053549955
 Start Angle: 165.96375653207352 End Angle: 296.565051177078


In [14]:
import cv2
import numpy as np
import math

"""
In case the angle between the joints is less than 180
"""

# Define the points for the two lines
line1_start = (100, 350)
line1_end = (300, 300)
line2_start = (200, 100)
line2_end = (300, 300)

# Create a blank image
image = np.zeros((500, 500, 3), dtype=np.uint8)

# Draw the two lines
cv2.line(image, line1_start, line1_end, (255, 0, 0), 2)
cv2.line(image, line2_start, line2_end, (0, 255, 0), 2)

# Calculate angles of the two lines relative to the horizontal
angle1 = math.atan2(line1_start[1] - line1_end[1], line1_start[0] - line1_end[0])
angle2 = math.atan2(line2_start[1] - line2_end[1], line2_start[0] - line2_end[0])
# Convert angles from radians to degrees
angle1_deg = np.degrees(angle1)
angle2_deg = np.degrees(angle2)
print(angle1, angle2, angle1_deg, angle2_deg)

# Ensure the angles are positive and lie within 0 to 360 degrees
# angle1_deg = angle1_deg % 360
# angle2_deg = angle2_deg % 360

# Calculate the internal angle between the two lines
arc_angle = abs(angle2_deg - angle1_deg)
print("arc_angle: ", arc_angle)
if arc_angle > 180:
    arc_angle = 360 - arc_angle  # Use the smaller angle

# Define the center of the arc (intersection point of the lines)
center = line1_end  # Intersection at (300, 300)

# Define radius for the arc
radius = 50

# Determine the start and end angles for the arc
start_angle = max(angle1_deg, angle2_deg)
end_angle = start_angle + arc_angle
print(f" Start Angle: {start_angle} End Angle: {end_angle}")

# Draw the arc
cv2.ellipse(image, center, (radius, radius), 0, start_angle, end_angle, (0, 255, 255), 2)

# Show the image
cv2.imshow('Arc between lines', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


2.896613990462929 -2.0344439357957027 165.96375653207352 -116.56505117707799
arc_angle:  282.5288077091515
 Start Angle: 165.96375653207352 End Angle: 243.434948822922


In [4]:
import cv2
import numpy as np
import math

def calculate_angle(a, b, c):
    """
    Calculate the angle between three points.
    Args:
    a, b, c: tuples of (x, y) coordinates
    Returns:
    Angle in degrees
    """
    ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    return ang + 360 if ang < 0 else ang

def draw_angle_arc(image, point1, point2, point3, color=(0, 255, 255), radius=50):
    """
    Draw an arc representing the angle between three points.
    Args:
    image: numpy array, the image to draw on
    point1, point2, point3: tuples of (x, y) coordinates
    color: tuple of (B, G, R) values for the arc color
    radius: radius of the arc
    """
    # Calculate the angle
    angle = calculate_angle(point1, point2, point3)
    
    # Calculate the start and end angles for the arc
    start_angle = math.atan2(point1[1] - point2[1], point1[0] - point2[0])
    end_angle = math.atan2(point3[1] - point2[1], point3[0] - point2[0])
    
    # Convert angles to degrees
    start_angle = math.degrees(start_angle)
    end_angle = math.degrees(end_angle)
    
    # Ensure the arc is drawn in the correct direction
    if start_angle > end_angle:
        start_angle, end_angle = end_angle, start_angle
    
    # Draw the arc
    cv2.ellipse(image, point2, (radius, radius), 0, start_angle, end_angle, color, 2)
    
    # Add text to display the angle
    cv2.putText(image, f"{angle:.1f}", 
                (point2[0] - 30, point2[1] - 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

# Create a blank image
image = np.zeros((500, 500, 3), dtype=np.uint8)

# Define points for different scenarios
scenarios = [
    [(100, 350), (300, 300), (200, 100)],  # Original scenario
    [(400, 100), (300, 300), (100, 200)],  # Facing right
    [(100, 200), (300, 300), (400, 100)],  # Facing left
    [(200, 400), (300, 300), (400, 200)],  # Upside down
]

# Draw lines and arcs for each scenario
for i, (point1, point2, point3) in enumerate(scenarios):
    # Draw lines
    cv2.line(image, point1, point2, (255, 0, 0), 2)
    cv2.line(image, point2, point3, (0, 255, 0), 2)
    
    # Draw angle arc
    draw_angle_arc(image, point1, point2, point3)
    
    # Label the scenario
    cv2.putText(image, f"Scenario {i+1}", 
                (point2[0] + 10, point2[1] + 40), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Show the image
    cv2.imshow(f'Angle Scenarios {i+1}', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
import cv2
import numpy as np
import math

def calculate_angle(a, b, c):
    """
    Calculate the angle between three points.
    Args:
    a, b, c: tuples of (x, y) coordinates
    Returns:
    Angle in degrees
    """
    ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    return ang + 360 if ang < 0 else ang

def draw_interior_angle_arc(image, point1, point2, point3, color=(0, 255, 255), radius=50):
    """
    Draw an arc representing the interior angle between three points.
    Args:
    image: numpy array, the image to draw on
    point1, point2, point3: tuples of (x, y) coordinates
    color: tuple of (B, G, R) values for the arc color
    radius: radius of the arc
    """
    # Calculate the angle
    angle = calculate_angle(point1, point2, point3)
    
    # Calculate the start and end angles for the arc
    start_vector = (point1[0] - point2[0], point1[1] - point2[1])
    end_vector = (point3[0] - point2[0], point3[1] - point2[1])
    
    start_angle = math.atan2(start_vector[1], start_vector[0])
    end_angle = math.atan2(end_vector[1], end_vector[0])
    
    # Convert angles to degrees
    start_angle = math.degrees(start_angle)
    end_angle = math.degrees(end_angle)
    
    # Ensure we're drawing the interior angle arc
    if abs(end_angle - start_angle) > 180:
        if start_angle < end_angle:
            start_angle += 360
        else:
            end_angle += 360
    
    # Draw the arc
    cv2.ellipse(image, point2, (radius, radius), 0, start_angle, end_angle, color, 2)
    
    # Add text to display the angle
    cv2.putText(image, f"{angle:.1f}", 
                (point2[0] - 30, point2[1] - 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

# Define points for different scenarios
scenarios = [
    [(458, 232), (518, 199), (571, 144)],  # Your specific scenario
    [(100, 350), (300, 300), (200, 100)],  # Original scenario
    [(400, 100), (300, 300), (100, 200)],  # Facing right
    [(100, 200), (300, 300), (400, 100)],  # Facing left
    [(200, 400), (300, 300), (400, 200)],  # Upside down
]

# Process each scenario
for i, (point1, point2, point3) in enumerate(scenarios):
    # Create a new blank image for each scenario
    image = np.zeros((600, 600, 3), dtype=np.uint8)
    
    # Draw lines
    cv2.line(image, point1, point2, (255, 0, 0), 2)
    cv2.line(image, point2, point3, (0, 255, 0), 2)
    
    # Draw interior angle arc
    draw_interior_angle_arc(image, point1, point2, point3)
    
    # Label the scenario
    cv2.putText(image, f"Scenario {i+1}", 
                (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Show the image
    cv2.imshow(f'Angle Scenario {i+1}', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()